## Applying mapper to plant gene expression data

In this notebook, we apply the mapper algorithm to the gene expression data. 
We assume that the directory structure is the same as it is in the repository. 
If not, you may have to check the paths and filenames to ensure the code runs 
without errors.

### import packages
First thing, import all the necessary packages and methods. Most of these are commonly used python packages, that can be easily installed. The only uncommon package is `kmapper` (for: *kepler mapper*), which can be pip installed. For more information and documentation, check out [this link](https://kepler-mapper.scikit-tda.org/en/latest/). We also need to import custom functions defined in the two files:
- `helper_functions.py` and 
- `lenses.py`

As the name suggests, one contains several utility functions required to perform tasks such as loading and scaling input data. The other contains the function to compute the *lens* (more on that further down in the notebook).

In [ ]:
import os
import numpy as np
import pandas as pd
import webbrowser
import pickle

# Clustering, Scaling
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

# keppler mapper
import kmapper as km

# Helper functions
from helper_functions import loaddata, get_color_dict, create_pie_chart_json
from lenses import fsga_transform

### set paths, specify file names
Next thing we want to do is set the paths to the directories containing `code`, `data`, and `results`. We also specify input files. The data to be analyzed is stored in two csv files. `clean_metadata.csv` contains the metadata, `clean_RNAseq_OutlierRemoved.csv` contains the gene expression data. 

The metadata file contains one sample per row, identified by its *SRA*. There are 3172 samples, and for each sample we have 7 descriptive attributes. We are particularly interested in three of them: namely, its plant *family*, *tissue* type and *stress* type. There are 16 different plant families, 8 tissue types and 10 stress types (including *healthy*) represented in the data. The RNAseq file, as the name suggests contains the gene expression for each sample, across a 6335 orthogroups. 

The third file, named `colors.pkl`, is a pickle file (that's a python specific file format!) that contains a mapping between colors (RGB and hex codes) and the different classes of the factors (*family*, *stress* and *tissue* type). Kepler mapper outputs an `html` file visualized in a browser window, other types of plots and figures are created using `seaborn` and `matplotlib`. We have set the color mapping in a file to ensure consistency across these plotting tools used in the project.

In [ ]:
projdir = "../.."
datadir = projdir + "/data"
resdir = projdir + "/results"
codedir = projdir + "/code"

factorfile = datadir + "/clean_metadata.csv"
colorfile = datadir + "/colors.pkl"
rnafile = datadir + "/raw_RNAseq_OutlierRemoved.csv"

### Loading data

The input data (metadata and gene expression) is stored in two separate files, but samples can be matched using the *SRA*. The function `loaddata` does exactly that. It loads both files into pandas dataframes and merges them using the *SRA* as the key. We perform an inner join, to keep only those *SRAs* for which we have metadata as well as gene expression. The `get_color_dict` is another utility function which looks for an existing `color.pkl` file and loads it. If the file is not present, the function will create it and return a dictionary containing the color mappings.

In [ ]:
factors = ["stress", "tissue", "family"]
df, orthos = loaddata(factorfile, rnafile, factors)
color_dict = get_color_dict(df, factors, colorfile)

### Set factor and level for analysis

Here, we set the factor, and the corresponding factor level that will be used to construct the lens function. We experimented with the following three combinations:
- filterfactor: *stress*, filterlevel: *healthy*
- filterfactor: *tissue*, filterlevel: *root*
- filterfactor: *tissue*, filterlevel: *leaf*

These variables are also used to construct the names of files and directories in which the results will be saved.

In [ ]:
filterfactor, filterlevel = ("stress", "healthy")
figdir = resdir + f"/{filterfactor}_{filterlevel}_lens_figures"
os.makedirs(figdir, exist_ok=True)

### Applying mapper

The first step in applying mapper is to initialize the mapper object and set some of its attributes like the clustering algorithms and the metric used by the clustering algorithm. We will be using *DBSCAN* which is the most commonly used clustering algorithm for mapper. The metric will be the correlation distance between sample gene expressions.

In [ ]:
# Initialize mapper object
mymapper = km.KeplerMapper(verbose=1)

# Define Nerve
nerve = km.GraphNerve(min_intersection=1)

# Define clustering algorithm
clust_metric = 'correlation'
clusterer = DBSCAN(metric=clust_metric)

### Define lens / filter function

Next, we need to define the *lens*. In the python file `lenses.py`, We have defined a function called *fsga_transform*. Given the filterfactor and the filterlevel of that factor (specified in the cell above), we construct a lens following the method described in [Nicolau et. al. 2011](https://www.pnas.org/content/108/17/7265).

For example, for filterfactor: *stress*, and filterlevel: *healthy*, we take the gene expression profiles of all the *healthy* samples from the data and fit a linear model to obtain an idealized *healthy* gene expression profile. Then we project all the samples on to this linear model and compute the residuals. The *lens* is the norm of the residual vector, which measures how much a sample gene expression deviates from the *healthy* one.

Here, we compute the lens and scale it using the standard *MinMaxScaler* from scikit-learn.

In [ ]:
# Define lens
scaler = MinMaxScaler()
residual, _, _ = fsga_transform(df, orthos, filterfactor, filterlevel)
lens = mymapper.project(residual, projection="l2norm", scaler=scaler)

### Define cover

Next step, defining a cover. We only need to specify the number of intervals (*cubes*) and the amount of overlap between consecutive intervals (*overlap*) and let kepler mapper take care of computing the cover. The values set in the cell block below are the ones we settled on after some experimentation. Feel free to change both the parameters but keep in mind that overlap must be between 0 and 100. Also, increasing the number of intervals will make the algorithm run slower so don't increase it beyond 150 or so.

In [ ]:
# Define cover
if filterfactor == "stress":
    cubes, overlap = (110, 75)
elif filterfactor == "tissue":
    if filterlevel == "root":
        cubes, overlap = (110, 90)
    elif filterlevel == "leaf":
        cubes, overlap = (140, 90)
    else:
        print("filterlevel must be root or leaf")
else:
    print("filterfactor must be stress or tissue")

cover = km.cover.Cover(n_cubes=cubes, perc_overlap=overlap/100.)

### Construct the mapper graph

With all the components required to construct the mapper graph ready, we can go ahead and call the *map* method of the KepplerMapper object which constructs the mapper graph. Keep an eye on the number of hypercubes, nodes and edges reported by the algorithm. Changing the cover parameters will change the graph size, making it denser or sparser. The computed graph is returned in form of a dictionary, we save it in a pickle file and use it later.

In [ ]:
# Create mapper graph with nodes, edges and meta-information.
graph = mymapper.map(lens=lens,
                     X=df[orthos],
                     clusterer=clusterer,
                     cover=cover,
                     nerve=nerve,
                     precomputed=False,
                     remove_duplicate_nodes=True)

graph["lens"] = lens.tolist()
fname = f"{filterfactor}_{filterlevel}_mapper_cubes_{cubes}_overlap_{overlap}"
fname = datadir + "/saved_mapper_graphs/" + fname + ".pkl"
with open(fname, "wb") as fp:
    pickle.dump(graph, fp, protocol=pickle.HIGHEST_PROTOCOL)

### Visualize the mapper graph

Lastly, we create the visualization, save it as an `html` file. Before we create the visualization, we add a couple of components to it. We will color the nodes of the mapper graph using the lens function values. The actual node color is determined averaging the colors of all samples in the corresponding cluster. We will also create tooltip that shows the factor values and lens values for samples in each cluster. If you uncomment the last line, Jupyter will also open the newly created visualization in a new browser tab.

In [ ]:
# Create dataframe column for tooltip
df['tooltips'] = [f'({str(p[0])}, {str(p[1])})' 
                    for p in zip(df[filterfactor], lens)]

# Specify file to save html output
fn = f"Filter_{filterfactor}_{filterlevel}_Cubes_{cubes}_Overlap_{overlap}.html"
fpath = figdir + '/' + fn
figtitle = f"Filter: {filterfactor}-{filterlevel},\
             #Cubes {cubes}, Overlap: {overlap/100.0}%"

_ = mymapper.visualize(graph,
                       path_html=fpath,
                       title=figtitle,
                       color_values=lens,
                       color_function_name="{}_lens".format(filterfactor),
                       custom_tooltips=df["tooltips"])

# # Uncomment to open mapper graph in browser (opens in default browser)
# webbrowser.open(fpath)

### Mapper with pie-chart nodes

The nodes of the mapper graph are in fact clusters of samples. In this next part, we want to visualize these clusters in the form of pie-charts. We will replace each node by a pie-chart that shows the distribution of node memebrs with respect to a given factor. The function `create_pie_chart_json` computes the corresponding pie-charts for each node of the mapper graph and stores the information in a json file. This json file is then pulled by the javascript code to create the visualization.

In [ ]:
json_dir = codedir + "/kepler_mapper_pie_nodes/json_data"
_ = create_pie_chart_json(graph, df, factors,
                          filterfactor, filterlevel, color_dict, json_dir)

### Visualizing the mapper with pie-chart nodes

After the json files are created and saved, open a terminal and go to the `code/kepler_mapper_pie_nodes` directory under the project root. Then run `python -m http.server` and once the server is running, open `http://0.0.0.8000` or `localhost:8000` in your browser. This will open up a directory listing with several html files. The filesnames follow the following pattern: 
- `<factor>_mapper_<filterfactor>_<filterlevel>_lens.html`

Which means, the lens for the mapper graph was computed using *filterfactor*, and *filterlevel*, and the pie-charts show the distribution of *factor* for each node. For example, the file `family_mapper_tissue_leaf_lens.html` contains mapper graphs computed using a *tissue* lens measuring deviation from the idealized *leaf* expression and the pie charts show the distribution of plant *family* across node members.